In [1]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [3]:
def sort_by_date(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        try:
            date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
        except ValueError:
            date = datetime.strptime(row['date'], '%m-%Y').date()
        if date <= date1:
            df1 = df1.append(row, ignore_index=True)
        if date > date1 and date <= date2:
            df2 = df2.append(row, ignore_index=True)
        if date > date2 and date <= date3:
            df3 = df3.append(row, ignore_index=True)
        if date > date3:
            df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(documents, n_topics, n_top_words, models):
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        display_topics(nmf, tfidf_feature_names, n_top_words)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        display_topics(lda, tf_feature_names, n_top_words)

In [4]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_date(df)

In [39]:
# get unnecessary symbols to remove from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
print(symbols_to_del)

['3', '-', '`', '?', ';', '*', '4', '1', '{', '\n', '\xad', '2', '№', '+', '—', '0', '[', ']', '8', '/', '−', ':', '”', '·', '>', '´', '(', '„', "'", '«', '―', '‒', '9', '°', '@', '6', '‑', '_', '－', '"', '“', '$', '=', '<', '7', '.', '%', '5', '́', '&', '…', '»', '!', '‘', '#', '–', ')', '€', '！', '’', ',', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [47]:
for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [112]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [morph.parse(word)[0].normal_form for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend(['наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'между', 'год', 'вопрос', 'страна', 
                 'россия', 'российский', 'такой', 'государство', 'другой', 'также', 'рф', 'мид', 'сегодня', 'два', 'катерина',
                 'спецкор', 'дать', 'прокомментировать', 'интервью', 'федерация', 'лавров', 'сторона', 'российско', 'господин',
                 'рассказывать', 'министр', 'встреча', 'состояться', 'сайт', 'итог', 'официальный', 'глава', 'иностранный',
                 'дело', 'москва'])

print(stopwords)

['за', 'вот', 'для', 'только', 'можно', 'без', 'этот', 'когда', 'человек', 'так', 'уже', 'теперь', 'но', 'быть', 'время', 'тем', 'один', 'потом', 'ничто', 'ну', 'они', 'ты', 'из', 'мы', 'я', 'не', 'под', 'у', 'нет', 'о', 'сказать', 'тут', 'с', 'мочь', 'на', 'то', 'ни', 'она', 'ли', 'бы', 'очень', 'через', 'или', 'надо', 'здесь', 'при', 'после', 'чем', 'в', 'раз', 'вы', 'по', 'а', 'он', 'если', 'кто', 'тогда', 'как', 'потому', 'чтобы', 'и', 'где', 'что', 'до', 'тот', 'же', 'более', 'это', 'там', 'себя', 'который', 'есть', 'даже', 'ещё', 'от', 'к', 'весь', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'между', 'год', 'вопрос', 'страна', 'россия', 'российский', 'такой', 'государство', 'другой', 'также', 'рф', 'мид', 'сегодня', 'два', 'катерина', 'спецкор', 'дать', 'прокомментировать', 'интервью', 'федерация', 'лавров', 'сторона', 'российско', 'господин', 'рассказывать', 'министр', 'встреча', 'состояться', 'сайт', 'итог', 'официальный', 'глава', 'иностранный', 'дело

In [116]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords)
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords)

In [117]:
get_topics(sorted_dfs[0].loc[:, 'text_clean'], 33, 15, ['nmf']) # до 2008

NMF
Topic 0:
международный развитие мировой важный экономический новый решение интерес работа сотрудничество мир стать внешний отношение являться
Topic 1:
палестинский израильский израиль пна дорожный палестинец карты газа аббас палестино ближневосточный мирный посредник хамас квартета
Topic 2:
обсудить переговоры визит сотрудничество двусторонний договориться торгово президент отношение взаимодействие ход конкретный экономический развитие рамка
Topic 3:
русский язык языка культура православный церковь выставка рубежом русскоязычный культуры зарубежный культурный слово изучение преподаватель
Topic 4:
обсе смид организация организации безопасности безопасность реформа председательство словенский председатель обзор деятельность действующий документ полевой
Topic 5:
бдипч бюро наблюдатель наблюдение обсе электоральный мониторинговый избирательный выборы выбор мониторинг миссия получение обращать мониторинга
Topic 6:
соотечественник соотечественников рубежом конгресс русскоязычный диаспора

In [118]:
get_topics(sorted_dfs[1].loc[:, 'text_clean'], 33, 15, ['nmf']) # 2008 - 2013

NMF
Topic 0:
международный экономический важный развитие глобальный мировой работа сотрудничество отношение новый основа стать решение интерес всего
Topic 1:
оон резолюция сб косово совет безопасность миссия международный ливия генеральный косовский ливии судан член решение
Topic 2:
визит двусторонний переговоры обсудить сотрудничество ход республика взаимодействие рабочий торгово актуальный президент развитие отношений экономический
Topic 3:
южный осетия абхазия осетии республика закавказье югоосетинский абхазский женевский абхазии неприменение грузия безопасность граница дискуссия
Topic 4:
посольство гражданин консульский суд власть находиться сотрудник судебный ситуация необходимый информация задержать ребёнок связь таиланд
Topic 5:
американский сша вашингтон про клинтон госдепартамент госсекретарь снв обама президент ребёнок администрация государственный штат приёмный
Topic 6:
одкб коллективный заседание совет организация безопасность сессия организации договор смид член рассмотрет

In [120]:
get_topics(sorted_dfs[2].loc[:, 'text_clean'], 36, 15, ['nmf']) # 2013 - 2016

NMF
Topic 0:
международный развитие экономический работа интерес важный сотрудничество мировой регион усилие всего общий основа президент новый
Topic 1:
сирийский сирия сирии алеппо боевик вооружённый террорист оппозиция террористический джабхата район ан правительство женевский оппозиции
Topic 2:
украинский украина украины киев киевский юго власть восток украине расследование заявление действие обстрел призывать вооружённый
Topic 3:
выздоровление погибших соболезнование выражать осуждать пострадавшим террористический решительно скорый родной близкие желать погибнуть теракт результат
Topic 4:
визит двусторонний сотрудничество переговоры обсудить ход актуальный республика торгово повестка взаимодействие лавр посетить рабочий мнение
Topic 5:
сша американский вашингтон дж керри администрация госсекретарь госдепартамент против про санкция том какой говорить санкционный
Topic 6:
снг заседание содружество совет субъект проект участник содружества смид сотрудничество мероприятие внешнеполитич

In [123]:
get_topics(sorted_dfs[3].loc[:, 'text_clean'], 38, 15, ['nmf']) # С 2016

NMF
Topic 0:
какой хотеть того том самый западный иметь идти международный говорить политический либо подобный внимание отношение
Topic 1:
сша американский вашингтон госдепартамент американец санкция администрация санкционный против дж двусторонний давление госсекретарь дипломатический вашингтоне
Topic 2:
сирийский сирия сирии террорист гуманитарный провинция зона боевик район игил вооружённый алеппо идлиб правительственный населить
Topic 3:
украина украинский киев украины киевский украине власть минский закон порошенко язык донбасс донбассе миссия режим
Topic 4:
визит двусторонний переговоры актуальный республика обсудить региональный мнение рабочий лавр обмен повестка перспектива ход международный
Topic 5:
рад пожаловать уважаемый добро дорогой приветствовать москве возможность коллеги друзья президент ценить хороший визит двусторонний
Topic 6:
химический оружие озхий озхо применение запрещение оружия конвенция отравлять хан кзхий сирийский вещество кзхо химоружие
Topic 7:
участие ме